In [6]:
import tensorflow as tf
print(tf.__version__)

1.14.0


In [4]:
pip install tensorflow==2.0.0-beta1 

In [7]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

data = keras.datasets.imdb

(train_data,train_labels) , (test_data,test_labels) = data.load_data(num_words=88000)

print(train_data[1])

17465344/17464789 [==============================] - 0s 0us/step
[1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463, 4369, 5012, 134, 26, 4, 715, 8, 118, 1634, 14, 394, 20, 13, 119, 954, 189, 102, 5, 207, 110, 3103, 21, 14, 69, 188, 8, 30, 23, 7, 4, 249, 126, 93, 4, 114, 9, 2300, 1523, 5, 647, 4, 116, 9, 35, 8163, 4, 229, 9, 340, 1322, 4, 118, 9, 4, 130, 4901, 19, 4, 1002, 5, 89, 29, 952, 46, 37, 4, 455, 9, 45, 43, 38, 1543, 1905, 398, 4, 1649, 26, 6853, 5, 163, 11, 3215, 10156, 4, 1153, 9, 194, 775, 7, 8255, 11596, 349, 2637, 148, 605, 15358, 8003, 15, 123, 125, 68, 23141, 6853, 15, 349, 165, 4362, 98, 5, 4, 228, 9, 43, 36893, 1157, 15, 299, 120, 5, 120, 174, 11, 220, 175, 136, 50, 9, 4373, 228, 8255, 5, 25249, 656, 245, 2350, 5, 4, 9837, 131, 152, 491, 18, 46151, 32, 7464, 1212, 14, 9, 6, 371, 78, 22, 625, 64, 1382, 9, 8, 168, 145, 23, 4, 1690, 15, 16, 4, 1355, 5, 28, 6, 52, 154, 462, 33, 89, 78, 285, 16, 145, 95]


In [8]:
word_index = data.get_word_index()

word_index = {k:(v+3) for k,v in word_index.items()}

word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2   #Unknown
word_index["<UNUSED>"] = 3

rev_word_index = dict([(value,key) for (key,value) in word_index.items()])

train_data = keras.preprocessing.sequence.pad_sequences(train_data, value=word_index["<PAD>"],padding="post",maxlen=250)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, value=word_index["<PAD>"],padding="post",maxlen=250)


def decode_review(text):
  return " ".join([rev_word_index.get(i, "?") for i in text])

# print(decode_review(test_data[2]))
# print(len(decode_review(test_data[2])))

# print(len(test_data[8]), len(test_data[1]))
# print(test_data[2])
# MODELS

model = keras.Sequential()
model.add(keras.layers.Embedding(88000,16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16,activation="relu"))    #relu:Rectified Linear Unit
model.add(keras.layers.Dense(1,activation="sigmoid"))  #It will generates the output as 0 or 1

model.summary()

model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

x_test = train_data[:10000]
x_train = train_data[10000:]

y_test = train_labels[:10000]
y_train = train_labels[10000:]

fit_model = model.fit(x_train,y_train,epochs=20,batch_size=512,validation_data=(x_test,y_test),verbose=1)

results = model.evaluate(test_data,test_labels)

print(results)


test_review = test_data[0]
predict = model.predict([test_review])
print("Review: ")
print(decode_review(test_review))
print("Prediction : " + str(predict[0]))
print("Actual : " + str(test_labels[0]))

model.save("model.h5")

1646592/1641221 [==============================] - 0s 0us/step
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          1408000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 1,408,289
Trainable params: 1,408,289
Non-trainable params: 0
___________

In [9]:
model = keras.models.load_model("model.h5")


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
